In [11]:
import os
import ctypes
from pathlib import Path

# Use absolute path to acados (same as run.sh)
ACADOS_PATH = "/home/bb/Desktop/projects/noc/robotic-mpc/acados"
ACADOS_LIB_PATH = f"{ACADOS_PATH}/lib"

os.environ["ACADOS_SOURCE_DIR"] = ACADOS_PATH
os.environ["LD_LIBRARY_PATH"] = f"{ACADOS_LIB_PATH}:" + os.environ.get("LD_LIBRARY_PATH", "")

# Preload shared libraries (LD_LIBRARY_PATH set at runtime doesn't work for dynamic linker)
# Load in dependency order
for lib in ["libblasfeo.so", "libhpipm.so", "libqpOASES_e.so", "libacados.so"]:
    lib_path = f"{ACADOS_LIB_PATH}/{lib}"
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"✓ Loaded {lib}")
    except OSError as e:
        print(f"✗ Failed to load {lib}: {e}")

print(f"\nACADOS_SOURCE_DIR = {os.environ['ACADOS_SOURCE_DIR']}")

# Verify t_renderer exists
tera_path = Path(ACADOS_PATH) / "bin" / "t_renderer"
print(f"t_renderer exists: {tera_path.exists()} at {tera_path}")

✓ Loaded libblasfeo.so
✓ Loaded libhpipm.so
✓ Loaded libqpOASES_e.so
✓ Loaded libacados.so

ACADOS_SOURCE_DIR = /home/bb/Desktop/projects/noc/robotic-mpc/acados
t_renderer exists: True at /home/bb/Desktop/projects/noc/robotic-mpc/acados/bin/t_renderer


In [12]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[0]  # o [1] se serve
sys.path.append(str(PROJECT_ROOT))

from simulator import Simulator, SimulationManager
import numpy as np

BASE_SURFACE_CONFIG = np.array({'a': -0.1, 'b': 0.1, 'c': -0.01, 'd': 0.01, 'e': 0.01, 'f': 0.0})
# Define joint limits for plotting boundaries
JOINT_POSITION_LIMITS = np.array([np.pi, np.pi, np.pi, np.pi, np.pi, np.pi])
JOINT_VELOCITY_LIMITS = np.array([3.15, 3.15, 3.15, 3.15, 3.15, 3.15])
JOINT_ACCELERATION_LIMITS = np.array([10.0, 10.0, 10.0, 10.0, 10.0, 10.0])

CROSS_OVER_FREQUENCIES = np.array([228.9, 262.09, 517.3, 747.44, 429.9, 1547.76]) # Simone's Magic Formula
SAMPLING_TIME = np.pi / (5*np.max(1/CROSS_OVER_FREQUENCIES)) #Shannon's theorem

base_config = {
    'dt': 0.0005,
    'simulation_time': 8,
    'surface_limits': ((-1, -0.1), (-1, 1)),
    'surface_origin': np.array([0.0, 0.0, 0.0]),
    'surface_orientation_rpy': np.array([0.0, 0.0, 0.0]),
    #surface function coefficients for a paraboloid
    'qdot_0': np.array([0, 0, 0, 0, 0, 0]),
    'q_0': np.array([np.pi/3, -np.pi/3, np.pi/4, -np.pi/2, -np.pi/2, 0.0]),
    'wcv': np.array([228.9, 262.09, 517.3, 747.44, 429.9, 1547.76]),
    'q_min': np.array([-2*np.pi, -2*np.pi, -np.pi, -2*np.pi, -2*np.pi, -2*np.pi], dtype=float),
    'q_max': np.array([+2*np.pi, +2*np.pi, +np.pi, +2*np.pi, +2*np.pi, +2*np.pi], dtype=float),
    'qdot_min': np.array([-np.pi, -np.pi, -np.pi, -np.pi, -np.pi, -np.pi], dtype=float),
    'qdot_max': np.array([np.pi, np.pi, np.pi, np.pi, np.pi, np.pi], dtype=float),
    'scene': True,
    'prediction_horizon': 200,  # Default value
    'solver_options': { 
        'nlp_solver_type': 'SQP',
        'qp_solver': 'PARTIAL_CONDENSING_HPIPM'
    }
}


In [13]:
from simulator import Simulator

sim = Simulator(**base_config)
sim.scene.jupyter_cell()

URDF successfully loaded: /home/bb/Desktop/projects/noc/robotic-mpc/ur_description/urdf/ur5.urdf
nq = 6, ngeoms(col) = 7, ngeoms(vis) = 7
Pinocchio model and data successfully created.
qp_solver = PARTIAL_CONDENSING_HPIPM
nlp_solver_type = SQP
rm -f libacados_ocp_solver_six_dof_robot_35a49961.so
rm -f six_dof_robot_35a49961_cost/six_dof_robot_35a49961_cost_y_0_fun.o six_dof_robot_35a49961_cost/six_dof_robot_35a49961_cost_y_0_fun_jac_ut_xt.o six_dof_robot_35a49961_cost/six_dof_robot_35a49961_cost_y_0_hess.o six_dof_robot_35a49961_cost/six_dof_robot_35a49961_cost_y_fun.o six_dof_robot_35a49961_cost/six_dof_robot_35a49961_cost_y_fun_jac_ut_xt.o six_dof_robot_35a49961_cost/six_dof_robot_35a49961_cost_y_hess.o acados_solver_six_dof_robot_35a49961.o
cc -fPIC -std=c99   -O2 -I/home/bb/Desktop/projects/noc/robotic-mpc/acados/include -I/home/bb/Desktop/projects/noc/robotic-mpc/acados/include/acados -I/home/bb/Desktop/projects/noc/robotic-mpc/acados/include/blasfeo/include -I/home/bb/Desktop/pro

In [14]:
sim.run()

Running time: 0.000 s of 8 s
Running time: 0.005 s of 8 s
Running time: 0.010 s of 8 s
Running time: 0.015 s of 8 s
Running time: 0.020 s of 8 s
Running time: 0.025 s of 8 s
Running time: 0.030 s of 8 s
Running time: 0.035 s of 8 s
Running time: 0.040 s of 8 s
Running time: 0.045 s of 8 s
Running time: 0.050 s of 8 s
Running time: 0.055 s of 8 s
Running time: 0.060 s of 8 s
Running time: 0.065 s of 8 s
Running time: 0.070 s of 8 s
Running time: 0.075 s of 8 s
Running time: 0.080 s of 8 s
Running time: 0.085 s of 8 s
Running time: 0.090 s of 8 s
Running time: 0.095 s of 8 s
Running time: 0.100 s of 8 s
Running time: 0.105 s of 8 s
Running time: 0.110 s of 8 s
Running time: 0.115 s of 8 s
Running time: 0.120 s of 8 s
Running time: 0.125 s of 8 s
Running time: 0.130 s of 8 s
Running time: 0.135 s of 8 s
Running time: 0.140 s of 8 s
Running time: 0.145 s of 8 s
Running time: 0.150 s of 8 s
Running time: 0.155 s of 8 s
Running time: 0.160 s of 8 s
Running time: 0.165 s of 8 s
Running time: 

NameError: name 't' is not defined